In [1]:
%load_ext autoreload
%autoreload 2

!pip install "gymnasium[accept-rom-license, atari]"

In [2]:
import tensorflow as tf
from replay_buffer import ReplayBuffer
from utils import update_q_network, polyak_averaging, get_small_dqn, get_standard_dqn
from env_sampler import ENV_SAMPLER
import numpy as np
import tqdm
import pandas as pd
import os

2023-06-10 23:27:10.262806: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-10 23:27:10.329017: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-10 23:27:10.329808: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 23:27:11.352654: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
GAMMA = 0.98
TAU = 0.01
BATCHSIZE = 32
N_BATCHES = 20
N_MULTI_ENVS = 64
EXPLORATION_RATE = 0.2

N_NEW_SAMPLES = 1000
N_SAMPLE_SETS = 3

CHECKPOINT_FREQ = 4
LOG_FREQ = 2

SAVE_TO_PATH = "ckpts/"
LOG_PATH = "logs/{}.json"

os.makedirs(SAVE_TO_PATH, exist_ok= True)
os.makedirs(LOG_PATH, exist_ok= True)

In [4]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

q_net = get_small_dqn()

target_net = tf.keras.models.clone_model(q_net)

env_sampler = ENV_SAMPLER(q_net, N_MULTI_ENVS)
replay_buffer = ReplayBuffer()

checkpoint = tf.train.Checkpoint(q_net = q_net, target_net = target_net)

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


Test Polyiak Averging

In [5]:
initial = q_net.trainable_variables[0].numpy()
q_net.trainable_variables[0].assign(np.ones_like(initial))
target_net.trainable_variables[0].assign(np.zeros_like(initial))

polyak_averaging(target_net, q_net, 0.2)
target_net.trainable_variables[0].numpy()

array([[[[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]],

        [[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]],

        [[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]]],


       [[[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]],

        [[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]],

        [[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.

In [6]:
%%timeit

polyak_averaging(target_net, q_net, 0.2)

845 µs ± 107 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
for _ in tqdm.tqdm(range(N_SAMPLE_SETS)):

    new_samples = env_sampler.sample(N_NEW_SAMPLES, epsilon = 1)
    replay_buffer.add_new_trajectory(new_samples)

100%|██████████| 3/3 [00:09<00:00,  3.20s/it]


In [8]:
%%timeit
ds = replay_buffer.generate_tf_dataset(N_BATCHES, BATCHSIZE)

89 ms ± 4.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
ds = replay_buffer.generate_tf_dataset(N_BATCHES, BATCHSIZE)

for _ in ds:
    
    pass

2023-06-10 23:27:35.852651: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [640]
	 [[{{node Placeholder/_3}}]]


In [10]:
for data in tqdm.tqdm(ds):
    
    update_q_network(data, q_net, target_net, optimizer, GAMMA)


100%|██████████| 20/20 [00:09<00:00,  2.03it/s]


In [11]:
results = env_sampler.measure_model_perforamnce(100, GAMMA, target_net)
names = ["average_rewards", "average_termination", "average_q_val", "average_target_q_val", "l2_diff",]
results = pd.Series(results, names)

Eval Model: 100%|██████████| 100/100 [00:40<00:00,  2.45it/s]


In [12]:
outer_steps = 0


while True:
    
    outer_steps +=1
    
    ds = replay_buffer.generate_tf_dataset(N_BATCHES, BATCHSIZE)
    
    bar = tqdm.tqdm(ds)
    
    losses = []

    bar = tqdm.tqdm(ds)

    for batch in bar:

        loss = update_q_network(data, q_net, target_net, optimizer, GAMMA)
        
        
        losses.append(loss)
        mean = float(np.mean(losses))
        bar.set_description(f"Loss {mean:.6f}")

    new_samples = env_sampler.sample(N_NEW_SAMPLES, epsilon=1)
    replay_buffer.add_new_trajectory(new_samples)
    polyak_averaging(target_net, q_net, TAU)
    
    
    if outer_steps % LOG_FREQ == 0:
        
        print("Logging Model Metrics")
        
        results = env_sampler.measure_model_perforamnce(100, GAMMA, target_net)
        names = ["average_rewards", "average_termination", "average_q_val", "average_target_q_val", "l2_diff",]
        results = pd.Series(results, names)
        
        results["average_loss"] = np.mean(losses)
        
        results.to_json(LOG_PATH.format(outer_steps))
        
        print(results)
        
        
    if outer_steps % CHECKPOINT_FREQ == 0:
        
        print("Saving Checkpoint")
        
        checkpoint.save(SAVE_TO_PATH)
        
        


  0%|          | 0/20 [00:00<?, ?it/s]
2023-06-10 23:28:27.490858: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [640]
	 [[{{node Placeholder/_3}}]]
Loss 0.010932: 100%|██████████| 20/20 [00:08<00:00,  2.47it/s]


Logging Model Metrics


Eval Model: 100%|██████████| 100/100 [00:45<00:00,  2.21it/s]


average_rewards         0.000000
average_termination          NaN
average_q_val           1.000000
average_target_q_val    2.350118
l2_diff                 1.350122
average_loss            0.010932
dtype: float64
Saving Checkpoint


  0%|          | 0/20 [00:00<?, ?it/s]
2023-06-10 23:29:24.509380: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [640]
	 [[{{node Placeholder/_2}}]]
  0%|          | 0/20 [00:00<?, ?it/s]
2023-06-10 23:29:36.063710: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [640]
	 [[{{node Placeholder/_2}}]]
Loss 0.000375:  45%|████▌     | 9/20 [00:04<00:05,  2.07it/s]


KeyboardInterrupt: 